In [1]:
# Importing reqired libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn

In [39]:
import warnings

# Remove all warnings
warnings.filterwarnings("ignore")

In [13]:
# Creating a Simple Neural Network Architectuer for linear regression problem

In [57]:
class ANNnet(nn.Module):
    def __init__(self, input_size, hidden_1_shape, hidden_2_shape, hidden_3_shape, output_size):
        super(ANNnet, self).__init__()
        
        # Initilization weights and biases
        self.weight_hidden_1 = nn.init.xavier_uniform(torch.empty(input_size, hidden_1_shape, requires_grad=True))
        self.bias_hidden_1 = torch.empty(hidden_1_shape, requires_grad=True)
        
        self.weight_hidden_2 = nn.init.xavier_uniform(torch.empty(hidden_1_shape, hidden_2_shape, requires_grad=True))
        self.bias_hidden_2 = torch.empty(hidden_2_shape, requires_grad=True)
        
        self.weight_hidden_3 = nn.init.xavier_uniform(torch.empty(hidden_2_shape, hidden_3_shape, requires_grad=True))
        self.bias_hidden_3 = torch.empty(hidden_3_shape, requires_grad=True)
                
        self.weight_output = nn.init.xavier_uniform(torch.empty(hidden_3_shape, output_size, requires_grad=True))
        self.bias_output = torch.empty(output_size, requires_grad=True)
        
        # define layers
        self.first_hidden_layer = None
        self.second_hidden_layer = None
        self.third_hidden_layer = None
        self.final_output = None
        
                                                             
    def forward(self, x):
        # calculating for first hidden layer
        self.first_hidden_layer = torch.matmul(x, self.weight_hidden_1) + self.bias_hidden_1
        activation_first_hidden_layer = torch.relu(self.first_hidden_layer)
        
        # calculating for first hidden layer
        self.second_hidden_layer = torch.matmul(activation_first_hidden_layer, self.weight_hidden_2) + self.bias_hidden_2
        activation_second_hidden_layer = torch.relu(self.second_hidden_layer)
        
        # calculating for first hidden layer
        self.third_hidden_layer = torch.matmul(activation_second_hidden_layer, self.weight_hidden_3) + self.bias_hidden_3
        activation_third_hidden_layer = torch.relu(self.third_hidden_layer)
        
        # calculating for first hidden layer
        self.final_output = torch.matmul(activation_third_hidden_layer, self.weight_output) + self.bias_output
        
        return self.final_output
    

In [21]:
# Hyperparameters
input_size = 3
hidden_1 = 8
hidden_2 = 5
hidden_3 = 5
output_size = 1
learning_rate = 0.01
num_epochs = 10
batch_size = 64

In [22]:
# Creating a model instance

In [58]:
model = ANNnet(
    input_size = input_size,
    hidden_1_shape = hidden_1, 
    hidden_2_shape = hidden_2,
    hidden_3_shape = hidden_3,
    output_size = output_size
)